In [1]:
import mne
import numpy as np
import csv
from scipy.stats import skew, kurtosis
import pyeeg as p
from numpy import nan
import math

In [2]:
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

In [3]:
def mean_kurtosis(df):
    kurtosis_vals = kurtosis(df)
    return np.mean(kurtosis_vals)

In [4]:
def mean_skewness(df):
    skew_vals = skew(df)
    return np.mean(skew_vals)

In [5]:
def func_mean_hurst(df):
    hurst_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        h_val = p.hurst(col)
        hurst_vals.append(h_val)
    return np.mean(hurst_vals)

In [6]:
def mean_pfd(df):
    pfd_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        pfd_val = p.pfd(col)
        pfd_vals.append(pfd_val)
    return np.mean(pfd_vals)

In [7]:
def mean_hjorth_mob_comp(df):
    mob_vals = []
    comp_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        mob_col, comp_col = p.hjorth(col)
        mob_vals.append(mob_col)
        comp_vals.append(comp_col)
    return np.mean(mob_vals), np.mean(comp_vals)

In [39]:
# data = mne.io.read_raw_edf('Splitted Files/eeg_73/73.1.edf'.format(row+1), preload=True)
# df = data.to_data_frame()
# data = data[:][0]
# print(mean_variance(df))
# print(mean_kurtosis(df))
# print(mean_skewness(df))
# print("Hurst is: ", func_mean_hurst(df))
# print("PFD IS: ", mean_pfd(df))
# print(mean_hjorth_mob_comp(df))
# print(all_psd(data))
# print(mean_spectral_entropy(data))

Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/Splitted Files/eeg_73/73.1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
Converting "time" to "<class 'numpy.int64'>"...
[13.606937026962989, 0.6118381265337367, 0.3627591347345236, 0.30219012476244633, 0.032642405000582124]


In [9]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(data))

    # FORMULA from some online post
    # psd_vals = (1/(fs*N)) * xdft * xdft;

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];

    # print(psd_vals.shape)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(psd_vals[:,freq_ix])
        psd_vals_list.append(eeg_band_fft[band] * 1000000)
    return psd_vals_list

In [10]:
def mean_spectral_entropy(df):
    psd_vals = all_psd(df)
    power_ratio = []
    sum_psd_vals = sum_psd(df)
    for val in psd_vals:
        power_ratio.append(val/sum_psd_vals)
    bands = [0,4,8,12,30,45]
    Fs = 256
    spec_entropy = p.spectral_entropy(df, bands, Fs, power_ratio)
    return spec_entropy

In [11]:
def sum_psd(df):
    psd_vals = all_psd(df)
    return np.sum(psd_vals)

In [74]:
with open('Combined-AfterMid/eeg74-0-20-combined.csv', 'w') as file:
    writer = csv.writer(file)
    for row in range(20):
        row_to_add = []
        data = mne.io.read_raw_edf('RecentSignals/eeg_74(0-20)/{}.edf'.format(row+1), preload=True)
        df = data.to_data_frame()
        data = data[:][0]
        
        power_vals = all_psd(data)
        mean_var = mean_variance(df)
        mean_skew = mean_skewness(df)
        mean_spec_ent = mean_spectral_entropy(data)
        mean_h_mob, mean_h_comp = mean_hjorth_mob_comp(df)
        mean_hurst_val = func_mean_hurst(df)
        mean_k = mean_kurtosis(df)
        
        row_to_add.append(power_vals[0])
        row_to_add.append(power_vals[1])
        row_to_add.append(power_vals[2])
        row_to_add.append(power_vals[3])
        row_to_add.append(power_vals[4])
        row_to_add.append(mean_skew)  
        row_to_add.append(mean_var)
        row_to_add.append(mean_k)  
        row_to_add.append(mean_h_mob)  
        row_to_add.append(mean_h_comp)
        row_to_add.append(mean_spec_ent)
        row_to_add.append(mean_hurst_val)  
        writer.writerow(row_to_add)

Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/RecentSignals/eeg_74(0-20)/1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
Converting "time" to "<class 'numpy.int64'>"...
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/RecentSignals/eeg_74(0-20)/2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
Converting "time" to "<class 'numpy.int64'>"...
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/RecentSignals/eeg_74(0-20)/3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 255  =      0.000 ...     0.996 secs...
Converting "time" to "<class 'numpy.int64'>"...
Extracting EDF parameters from /home/anjali/Desktop/Mehnat/BTP2/RecentSignals/eeg_74(0-20)/4.edf...
EDF file detected
Setting channel info struct